## Transcripción de voz a texto en tiempo real

In [2]:
# python version == 3.8.20

!pip install pyaudio
!pip install sounddevice
!pip install torch
!pip install webrtcvad
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import sounddevice as sd
import numpy as np
import whisper
import queue
import pyaudio
import wave
import webrtcvad

# Configuración de PyAudio
#CHUNK = 4096    #1024              # Tamaño de cada fragmento de audio
FORMAT = pyaudio.paInt16  # Formato de audio
CHANNELS = 1              # Audio mono
RATE = 16000 #11025              # Frecuencia de muestreo compatible con Whisper
DURATION = 0.03
BYTES_PRE_SAMPLE = 1
chunk = DURATION * RATE  
chunk = int(chunk)
# Inicializar PyAudio
audio = pyaudio.PyAudio()
# Inicializar cola para audio
audio_queue = queue.Queue()

# Crear el modelo de Whisper
model = whisper.load_model("tiny")  # Cambiar el modelo según sea necesario
# tiny 1Gb 10x
# base 1Gb 7x
# small 2Gb 4x
# medium 5Gb 2x
# large 10Gb 1x
# turbo 6Gb 8x

# Inicializar VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Nivel de sensibilidad: 0 (menos sensible) a 3 (más sensible)

# Función para detectar actividad de voz
def is_speech(data, sample_rate):
#    print("checking if is speech...")
    return vad.is_speech(data, sample_rate)

# Función para reproducir audio
def play_audio(data):
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(data)
    stream.stop_stream()
    stream.close()

# Función para capturar audio en tiempo real
def audio_callback(indata, frames, time, status):
    if status:
        print(f"Status: {status}")
    audio_queue.put(indata.copy())

print("Dispositivos de audio disponibles:")
for i in range(audio.get_device_count()):
    info = audio.get_device_info_by_index(i)
    print(f"ID: {i}, Nombre: {info['name']}, Entrada: {info['maxInputChannels']}, Salida: {info['maxOutputChannels']}")


ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


Dispositivos de audio disponibles:
ID: 0, Nombre: HDA Intel HDMI: 0 (hw:0,3), Entrada: 0, Salida: 8
ID: 1, Nombre: HDA Intel HDMI: 1 (hw:0,7), Entrada: 0, Salida: 8
ID: 2, Nombre: HDA Intel HDMI: 2 (hw:0,8), Entrada: 0, Salida: 8
ID: 3, Nombre: HDA Intel PCH: ALC3239 Analog (hw:1,0), Entrada: 2, Salida: 0
ID: 4, Nombre: hdmi, Entrada: 0, Salida: 8
ID: 5, Nombre: pulse, Entrada: 32, Salida: 32
ID: 6, Nombre: default, Entrada: 32, Salida: 32


In [4]:
# Opcion 1.- Real-Time con sounddevice
with sd.InputStream(samplerate=RATE, channels=CHANNELS, callback=audio_callback):
    print("Grabando... Presiona Ctrl+C para detener.")
    try:
        # Acumular audio y transcribir en tiempo real
        while True:
            audio_data = audio_queue.get()
            # Convertir audio a un formato que Whisper entienda
            audio_np = np.squeeze(audio_data)
            result = model.transcribe(audio_np, language="es")
            print("Transcripción: ", result["text"])
    except KeyboardInterrupt:
        print("Grabación detenida.")
    except Exception as e:
        print(f"Error: {e}")


PortAudioError: Error querying device -1

In [7]:
# Opcion 2.- Real-Time con pyaudio
# transcripción contínua periódica
# Abrir el stream para capturar el audio del micrófono
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS, 
                    rate=RATE, 
                    input=True, 
                    frames_per_buffer=chunk)
print("Grabando... Presiona Ctrl+C para detener.")

try:
    audio_buffer = b''  # Buffer para almacenar audio capturado
    while True:
        # Leer audio desde el micrófono
        data = stream.read(chunk, exception_on_overflow=False)
        audio_buffer += data

        # Procesar el audio cada 5 segundos aproximadamente
        if len(audio_buffer) >= RATE * 5:
            # Convertir a formato que Whisper entienda
            audio_np = np.frombuffer(audio_buffer, dtype=np.int16).astype(np.float32) / 32768.0

            # Transcribir el audio
            result = model.transcribe(audio_np, language="es")
            print("Transcripción: ", result["text"])

            # Reproducir el audio capturado
            play_audio(audio_buffer)

            # Limpiar el buffer después de procesar
            audio_buffer = b''

except KeyboardInterrupt:
    print("Grabación detenida.")
except Exception as e:
    print(f"Error: {e}")

# Cerrar streams y PyAudio
stream.stop_stream()
stream.close()
audio.terminate()


Grabando... Presiona Ctrl+C para detener.
Transcripción:  
Grabación detenida.


In [14]:
# opcion 3.- transcripción mediante detección activa de voz VAD

INPUT_DEVICE_ID = 1  # Cambiar según el dispositivo de entrada
OUTPUT_DEVICE_ID = 3  # Cambiar según el dispositivo de salida

# Abrir el stream para capturar el audio del micrófono
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS, 
                    rate=RATE, 
                    input=True, 
#                    output=False,
#                    input_device_index=INPUT_DEVICE_ID,
#                    output_device_index=OUTPUT_DEVICE_ID,
                    frames_per_buffer=chunk)

print("Esperando actividad de voz... Presiona Ctrl+C para detener.")

try:
    audio_buffer = b''  # Buffer para almacenar audio capturado
    speaking = False    # Indicador de actividad de voz
    while True:
        # Leer audio desde el micrófono
        data = stream.read(chunk, exception_on_overflow=False)
        
        # Detectar actividad de voz
        if is_speech(data, RATE):
            if not speaking:
                print("Usuario comenzó a hablar...")
                speaking = True
            audio_buffer += data
        else:
            if speaking:
                print("Usuario terminó de hablar.")
                speaking = False
                
                # Convertir el audio capturado a formato que Whisper entienda
                audio_np = np.frombuffer(audio_buffer, dtype=np.int16).astype(np.float32) / 32768.0
                
                # Transcribir el audio
                print("Transcribiendo...")
                result = model.transcribe(audio_np, language="es")
                print("Transcripción: ", result["text"])
                
                # Limpiar el buffer después de procesar
                audio_buffer = b''

except KeyboardInterrupt:
    print("Grabación detenida.")
except Exception as e:
    print(f"Error: {e}")

# Cerrar streams y PyAudio
stream.stop_stream()
stream.close()
audio.terminate()


Esperando actividad de voz... Presiona Ctrl+C para detener.
Usuario comenzó a hablar...
Usuario terminó de hablar.
Transcribiendo...
Transcripción:   Gracias.
Usuario comenzó a hablar...
Usuario terminó de hablar.
Transcribiendo...
Grabación detenida.
